# Applications of SOS Programming in Flowpipe Construction

Flowpipe construction consists of under or over-approximating the sets of states reachable by dynamical systems. Recently a method has been developed for the class of polynomial ODEs with uncertain initial states (see [1], abbreviated `XFZ18under`). This method consists of reducing the Hamilton-Jacobi-Bellman equation to a hierarchy of semidefinite programs.

In this notebook we consider the problem of approximating the flowpipe of a system of polynomial ODEs using `XFZ18under`. This is a Julia implementation that relies on the JuMP ecosystem (`JuMP`, `PolyJuMP`, `SumOfSquares`, `MathProgInterface`, `MathOptInterfaceMosek`) and the `JuliaAlgebra` ecosystem (`MultivariatePolynomials`, `DynamicPolynomials`). The implementation is evaluated on a set of standard benchmarks from formal verification and control engineering domains.

---

**References:**

- [1] Xue, B., Fränzle, M., & Zhan, N. (2018, April). [Under-Approximating Reach Sets for Polynomial Continuous Systems. In Proceedings of the 21st International Conference on Hybrid Systems: Computation and Control (part of CPS Week) (pp. 51-60). ACM.](https://dl.acm.org/citation.cfm?id=3178133)

**Quick links to documentation:**

- www.juliaopt.org/SumOfSquares.jl/latest/
- https://sums-of-squares.github.io/sos/index.html

## Van-der-Pol system

Consider the following van-der-Pol system:

$$
\dot{x}_1 = x_2 \\
\dot{x}_2 = -0.2x_1 + x_2 - 0.2x_1^2 x_2
$$

In [2]:
using MultivariatePolynomials,
      JuMP,
      PolyJuMP,
      SumOfSquares,
      DynamicPolynomials,
      MathOptInterfaceMosek,
      MathematicalSystems,
      MosekTools
 
const ∂ = differentiate

┌ Info: Recompiling stale cache file /Users/forets/.julia/compiled/v1.1/MathOptInterfaceMosek/sqIeN.ji for MathOptInterfaceMosek [0087ddc6-3964-5e57-817f-9937aefb0357]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /Users/forets/.julia/compiled/v1.1/MathematicalSystems/6oLdk.ji for MathematicalSystems [d14a8603-c872-5ed3-9ece-53e0e82e39da]
└ @ Base loading.jl:1184


differentiate (generic function with 18 methods)

In [55]:
# symbolic variables
@polyvar x₁ x₂ t

# time duration (scaled, see dynamics below)
T = 1.0 

# dynamics
f = 2 * [x₂, -0.2*x₁ + x₂ - 0.2*x₁^2*x₂] 

# set of initial states X₀ = {x: V₀(x) <= 0}
V₀ = x₁^2 + x₂^2 - 0.25

# constraints Y = {x: g(x) >= 0} compact search space Y x [0, T]
g = 25 - x₁^2 - x₂^2

# degree of the relaxation
k = 4

# monomial vector up to order k, 0 <= sum_i alpha_i <= k, if alpha_i is the exponent of x_i
X = monomials([x₁, x₂], 0:k)
XT = monomials([x₁, x₂, t], 0:k)

# create a SOS JuMP model to solve with Mosek
model = SOSModel(with_optimizer(Mosek.Optimizer))

# add unknown Φ to the model
@variable(model, Φ, Poly(XT))

# jacobian
∂t = α -> ∂(α, t)
∂xf = α -> ∂(α, x₁) * f[1] + ∂(α, x₂) * f[2] 
LΦ = ∂t(Φ) + ∂xf(Φ)

# Φ(x, t) at time 0
Φ₀ = subs(Φ, t => 0.)

# scalar variable
@variable(model, ϵ)

dom1 = @set t*(T-t) >= 0 && g >= 0
dom2 = @set g >= 0
@constraint(model, ϵ >= 0.)
@constraint(model, LΦ ∈ SOSCone(), domain = dom1)
@constraint(model, ϵ - LΦ ∈ SOSCone(), domain = dom1)
@constraint(model, Φ₀ - V₀ ∈ SOSCone(), domain = dom2)
@constraint(model, ϵ + V₀ - Φ₀ ∈ SOSCone(), domain = dom2)

@objective(model, Min, ϵ)

ϵ

In [56]:
optimize!(model)

println("Relaxation order : k = $k")
println("JuMP.termination_status(model) = ", JuMP.termination_status(model))
println("JuMP.primal_status(model) = ", JuMP.primal_status(model))
println("JuMP.dual_status(model) = ", JuMP.dual_status(model))
println("JuMP.objective_bound(model) = ", JuMP.objective_bound(model))
println("JuMP.objective_value(model) = ", JuMP.objective_value(model))

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 199             
  Cones                  : 0               
  Scalar variables       : 36              
  Matrix variables       : 10              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 199             
  Cones               

### Comparison with a YALMIP implementation in MATLAB

|  Package    | k    |Constraints|Scalar variables|Matrix variables|Time(s)|
|-------------|------|-----------|----------------|----------------|-------|
|JuMP v0.18.5         |    2 |       245 |            175 |              8 |   < 1 |
|**JuMP v0.19.0**         |    2 |    83    |      13       |            8   |   < 1 |
|YALMIP       |    2 |       152 |             63 |              10|   < 1 |
||
|JuMP v0.18.5         |    3 |      893 |           715 |              10|   ~ 1 |
|**JuMP v0.19.0**         |    3 |    199    |        21     |        10       |   < 1 |
|YALMIP       |    3 |       254 |            121 |              10|   ~ 1 |
||
|JuMP v0.18.5         |    4 |      893 |           730 |              10|  ~1 |
|**JuMP v0.19.0**         |    4 |   199     |       36      |      10         |   < 1 |
|YALMIP       |    4 |       394 |           206  |              10|  1.18 |
||
|JuMP v0.18.5         |    5 |       2639 |            2309 |              10 |   1.17 |
|**JuMP v0.19.0**         |    5 |     387   |       57      |         10      |   < 1 |
|YALMIP       |    5 |      578  |         323     |          10    |  0.11 |
||
|JuMP v0.18.5         |    6 |       2639 |         2337   |              10|   0.90 |
|**JuMP v0.19.0**         |    6 |    387    |       85      |       10        |   < 1 |
|YALMIP       |    6 |    812    |         477    |    10          |  1.10  |
||
|JuMP v0.18.5         |    7 |      6725 |       6183     |              10|  5.62 |
|**JuMP v0.19.0**         |    7 |   663     |     121        |      10         |   < 1 |
|YALMIP       |    7 |     1102   |        673     |        10      |  1.52 |
||
|JuMP v0.18.5         |    8 |     6725  |      6228       |          10    |   6.06 |
|**JuMP v0.19.0**         |    8 |  663      |        166     |           10    |   < 1 |
|YALMIP       |    8 |    1454    |       916       |          10    |  1.10 |
||
|JuMP v0.18.5         |    9 |    15269   |      14447      |           10   |  41.2  |
|**JuMP v0.19.0**         |    9 |      1043  |        221     |        10       |   1.70 |
|YALMIP       |    9 |   1874     |       1211     |         10     |  1.58  |
||
|JuMP v0.18.5         |   10 |    15269   |     14513       |      10        |  38.1 |
|**JuMP v0.19.0**         |    10 |    1043    |    287         |     10          |   1.67 |
|YALMIP       |   10 |     2368   |       1563     |        10      |  2.73 |
||
|JuMP v0.18.5         |   11 |   31617    |     30439       |     10         | 244  |
|**JuMP v0.19.0**         |    11 |   1543     |      365       |     10          |   4.88 |
|YALMIP       |   11 |   2942    |       1977      |       10       |  2.30 |
||
|JuMP v0.18.5         |   12 |     31617  |     30530       |       10       | 231 |
|**JuMP v0.19.0**         |    12 | 1543       |   456          |         10      |  5.02 |
|YALMIP       |   12 |    3602    |      2458       |      10        | 6.57  |

Related: https://github.com/JuliaOpt/MosekTools.jl/issues/11.

## Extracting the under and over approximations

In [57]:
# Recovering the solution:
ϵopt = JuMP.objective_value(model)

# Punder <= 0
Punder = subs(JuMP.value(model[:Φ]), t => T)

-0.0021760503056437726x₁⁴ + 0.03237705635327642x₁³x₂ - 0.013315607202341661x₁²x₂² + 0.011033724936160602x₁x₂³ - 0.006590230901819379x₂⁴ - 9.839119379896526e-21x₁³ - 4.833341971297815e-20x₁²x₂ - 4.139050910683632e-20x₁x₂² + 2.0666922228154174e-22x₂³ + 0.27347582897153294x₁² - 0.8516247221359877x₁x₂ + 0.469737949698723x₂² + 2.9279424673129996e-25x₁ + 5.952860935439434e-19x₂ + 28.363527748437562

In [58]:
# Pover <= 0
Pover = subs(JuMP.value(model[:Φ]), t => T) - ϵopt * (T+1)

-0.0021760503056437726x₁⁴ + 0.03237705635327642x₁³x₂ - 0.013315607202341661x₁²x₂² + 0.011033724936160602x₁x₂³ - 0.006590230901819379x₂⁴ - 9.839119379896526e-21x₁³ - 4.833341971297815e-20x₁²x₂ - 4.139050910683632e-20x₁x₂² + 2.0666922228154174e-22x₂³ + 0.27347582897153294x₁² - 0.8516247221359877x₁x₂ + 0.469737949698723x₂² + 2.9279424673129996e-25x₁ + 5.952860935439434e-19x₂ - 5.42176351702993

## Visualization

In [ ]:
using LinearAlgebra, StaticPolynomials

# filter out the smallest coefficients?
filter_coeffs = false

if filter_coeffs
    max_coeff = maximum(abs.(Punder.a))

    # plots are *very* sensitive to small values
    # make tol bigger than 0 to filter out some coefficients
    tol = 0.0 # max_coeff / 1e19
    kept_coeffs = map(c -> abs(c) > tol, Punder.a)
    Punder = dot(Punder.a[kept_coeffs], Punder.x[kept_coeffs])
    Pover = dot(Pover.a[kept_coeffs], Pover.x[kept_coeffs]);
end

# we now convert to a static polynomial for faster evaluation
Punder_st = StaticPolynomials.Polynomial(Punder)
Pover_st = StaticPolynomials.Polynomial(Pover);

_Punder(x, y) = Punder_st([x, y])
_Pover(x, y) = Pover_st([x, y])

In [ ]:
Punder_st

In [ ]:
sprint(print, Punder)

### Plots using ImplicitEquations

In [ ]:
using ImplicitEquations, Plots
gr()

In [ ]:
G = plot()
plot!(G, _Punder ⩵ 0., xlims=(-10, 10), ylims=(-10, 10), color="red")
plot!(G, _Pover ⩵ 0., xlims=(-10, 10), ylims=(-10, 10), color="blue")
G

In [ ]:
G = plot()
Gu = plot(_Punder ≪ 0., xlims=(-10, 10), ylims=(-10, 10))
Go = plot(_Pover ≪ 0., xlims=(-10, 10), ylims=(-10, 10))
plot(Gu, Go)

### Plots using IntervalConstraintProgramming

In [ ]:
using IntervalConstraintProgramming, ValidatedNumerics

In [ ]:
@function Funder(x₁, x₂) = _Punder(x₁, x₂)

In [ ]:
Bx₁x₂ = IntervalBox(-10..10, -10..10)

In [ ]:
Cunder = IntervalConstraintProgramming.@constraint Funder(x₁, x₂) <= 0.0 # no

In [ ]:
paving = pave(Cunder, Bx₁x₂)